In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
task_configs = {
    "COLA": {
        "lr": 1e-5,
        "l2": 0,
        "batch_size": 8,
        "n_epochs": 3
    },
#     "SST2": {
#         # TODO
#     },
#     "MNLI": {
#         "lr": 1e-5,
#         "batch_size": 2,
#         "l2": 0,
#         "n_epochs": 1
#     },
#     "WNLI": {
#         "lr": 1e-4,
#         "l2": 0,
#         "batch_size": 32,
#     },
#     "QQP": {
#         "lr": 1e-5,
#         "l2": 0,
#         "batch_size": 32
#     },
#     "MRPC": {
#         "lr": 1e-5,
#         "l2": 0,
#         "batch_size": 2
#     },
#     "STSB": {
#         "lr": 1e-5,
#         "l2": 0,
#         "n_epochs": 3,
#     },
#     "QNLI": {
#         # TODO
#     }
}

In [ ]:
import os
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.glue.glue_tasks import create_tasks
from metal.mmtl.trainer import MultitaskTrainer


for task_name, config in task_configs.items():
    writer_dict = {"writer": "tensorboard",
        "writer_config": {  # Log (or event) file stored at log_dir/run_dir/run_name
            "log_dir": f"{os.environ['METALHOME']}/logs",
            "run_dir": "BERT_base",
            "run_name": task_name,
            "writer_metrics": [],  # May specify a subset of metrics in metrics_dict to be written
            "include_config": True,  # If True, include model config in log
        },
    }
    

    tasks = create_tasks([task_name], 
                         dl_kwargs={'batch_size': config.get("batch_size", 32)})
    model = MetalModel(tasks)
    trainer = MultitaskTrainer(writer_dict)
    trainer.train_model(
        model,
        tasks,
        lr=config.get("lr", None),
        l2=config.get("l2", None),
        log_every = 1,
        checkpoint_metric="model/train/loss",
        checkpoint_metric_mode="min",
        checkpoint_best = True,
        n_epochs=config.get("n_epochs", None),
        progress_bar=True
    )

Loading COLA Dataset


Using GPU...

Network architecture:
MetalModel(
  (input_modules): ModuleDict(
    (COLA): BertEncoder(
      (bert_model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1)
                )
                (output): BertSelfOutput(
                  (dense): Linear(in_features=768, out_features=768

[1.0 epo]: TRAIN:[loss=0.516] VALID:[COLA/accuracy=0.791, COLA/matthews_corr=0.470]
Saving model at iteration 1.0 with best (min) score 0.516


[2.0 epo]: TRAIN:[loss=0.309] VALID:[COLA/accuracy=0.811, COLA/matthews_corr=0.529]
Saving model at iteration 2.0 with best (min) score 0.309
